### importing require packages

In [1]:
from __future__ import print_function

import json
import os
import numpy as np
import sys
import h5py

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.engine import Input
from keras.layers import Embedding, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.preprocessing import sequence
from embeddings import Embeddings
from keras.callbacks import ModelCheckpoint

from nltk.tokenize import word_tokenize

Using TensorFlow backend.


EMBEDDING(100,4,1,4) STARTED .....
Loading the embeddings from the cache
EMBEDDING(100,4,1,4) COMPLETED .....


## Instantiate Embeddings 

In [2]:
embeddings = Embeddings(100, 4, 1, 4)

Loading the embeddings from the cache


### getting data from preprocessing

In [3]:
word2vec_weights = embeddings.get_weights()
word2index, index2word = embeddings.get_vocabulary()
word2vec_model = embeddings.get_model()
tokenized_indexed_sentences = embeddings.get_tokenized_indexed_sentences()

### generating training data

In [4]:
window_size = 5
vocab_size = len(word2index)
print(vocab_size)
#sorted(window_size,reverse=True)
#sentence_max_length = max([len(sentence) for sentence in tokenized_indexed_sentence ])

42047


In [5]:
seq_in = []
seq_out = []
# generating dataset
for sentence in tokenized_indexed_sentences:
    for i in range(len(sentence)-window_size-1):
        x = sentence[i:i + window_size]
        y = sentence[i + window_size]
        seq_in.append(x)#[]
        seq_out.append(word2vec_weights[y])

# converting seq_in and seq_out into numpy array
seq_in = np.array(seq_in)
seq_out = np.array(seq_out)
n_samples = len(seq_in)
print ("Number of samples : ", n_samples)

Number of samples :  47295


## Defining model

In [9]:
# Changes to the model to be done here
model = Sequential()
model.add(Embedding(input_dim=word2vec_weights.shape[0], output_dim=word2vec_weights.shape[1], weights=[word2vec_weights]))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1024))
model.add(Dropout(0.2))
model.add(Dense(word2vec_weights.shape[1], activation='relu'))
#model.load_weights("../weights/Model-LSTM-2-Layers-512-50-Epochs/weights.09-0.20.hdf5")
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         4204700   
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 1024)        4608000   
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 1024)        0         
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 1024)        8392704   
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 1024)        0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 1024)              8392704   
_________________________________________________________________
dropout_7 (Dropout)          (None, 1024)              0         
__________

In [10]:
model_weights_path = "../weights/Model-LSTM-2-Layers-512-50-Epochs"
if not os.path.exists(model_weights_path):
    os.makedirs(model_weights_path)
checkpoint_path = model_weights_path + '/weights-02.{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

## Train Model

In [12]:
model.fit(seq_in[:3000], seq_out[:3000], epochs=2, verbose=1, validation_split=0.2, batch_size=128, callbacks=[checkpoint])

Train on 2400 samples, validate on 600 samples
Epoch 1/2
2400/2400 [==============================] - 75s - loss: 0.3851 - acc: 0.1067 - val_loss: 0.3928 - val_acc: 0.2267
Epoch 2/2
2400/2400 [==============================] - 76s - loss: 0.3649 - acc: 0.1400 - val_loss: 0.3892 - val_acc: 0.2283


### model predict

In [13]:
start = 97
pattern = list(seq_in[start])
print("\"",' '.join(index2word[index] for index in pattern))
for i in range(100):
    prediction = model.predict(np.array([pattern]))
    pred_word = word2vec_model.similar_by_vector(prediction[0])[0][0]
    sys.stdout.write(pred_word + " ")
    pattern.append(word2index[pred_word])
    pattern = pattern[1:len(pattern)]

" teams have been in the
full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full full 

In [14]:
pred_words = word2vec_model.similar_by_vector(prediction[0])
pred_words

[('full', 0.7538601160049438),
 ('boat', 0.7491844892501831),
 ('disobedience', 0.7489128708839417),
 ('later', 0.7480583190917969),
 ('prussian', 0.7434743642807007),
 ('nasa', 0.7433779835700989),
 ('coin', 0.740304708480835),
 ('50', 0.7396358251571655),
 ('televised', 0.7383438348770142),
 ('latest', 0.7376772165298462)]

In [19]:
from itertools import product

In [20]:
word_pairs = list(product([word2vec_model.similar_by_vector(seq_out[97])[0][0]], list(zip(*pred_words))[0]))

In [21]:
max([word2vec_model.similarity(w1, w2) for w1, w2 in word_pairs])

0.2680437971440815